Utils.jl:-

In [1]:
import Distributions
import SpecialFunctions

"""    gaussianMechConstant(ϵ::Real,δ::Real)
Compute the proportionality constant κ(ϵ,δ) for the Gaussian mechanism.
"""
function gaussianMechConstant(ϵ::Real, δ::Real)
    K = sqrt(2) * SpecialFunctions.erfinv(1-2δ)
    return (K+sqrt(K^2+2ϵ))/(2ϵ)
end

"""    (k₁, k₂) = gaussianMechConstant2(ϵ::Real,δ::Real)
Compute the proportionality constant κ(ϵ,δ) for the Gaussian mechanism,
both with the formula using the inverse Q-function (k₁), and according to the
approximation of Theorem A.1 in Dwork and Roth's book (k₂, higher value).
"""
function gaussianMechConstant2(ϵ::Real, δ::Real)
    K = sqrt(2) * SpecialFunctions.erfinv(1-2δ)
    return ((K+sqrt(K^2+2ϵ))/(2ϵ), sqrt(2log(1.25/δ))/ϵ)
end

"""    mean_dp(x::Vector,epsilon::Float64,delta::Float64=0.0)
Compute the average of a set of numbers contained in the vector x,
in an (ϵ,δ)-differentially private way. It is assumed that each entry
x_i of the vector is in [0,1], and the adjacency relation looks at
arbitrary variations in one entry x_i, within this interval.
"""
function mean_dp(x::Vector, ϵ::Real, δ::Real=0.0)
    if δ == 0.0
        d = Distributions.Laplace(0, 1/(ϵ*length(x)))
    else
        d = Distributions.Normal(0, gaussianMechConstant(ϵ,δ)/length(x))
    end
    return Distributions.mean(x) + Distributions.rand(d)
end

"""    laplaceMech(x,f,l1sens::Real,ϵ::Real)
Computes a randomized version to f(x) according to the
Laplace mechanism, for ϵ-differential privacy. l1sens is
the l1-sensitivity of f, which must be computed and provided by the user.
f must take values in R^k, for some k, i.e., return an array of k real values.
"""
function laplaceMech(x, f, l1sens::Real, ϵ::Real)
    t = f(x)
    d = Distributions.Laplace(0, l1sens/ϵ)
    return (t .+ Distributions.rand(d, length(t)))
end

"""    gaussianMech(x,f,l2sens::Real,ϵ::Real,δ::Real)
Computes a randomized version to f(x) according to the
Gaussian mechanism, for (ϵ,δ)-differential privacy. l2sens is
the l2-sensitivity of f, which must be computed by the user.
f must take values in R^k, from some k, i.e., return an array
of k real values.
"""
function gaussianMech(x, f, l2sens::Real, ϵ::Real, δ::Real)
    t = f(x)
    d = Distributions.Normal(0, gaussianMechConstant(ϵ, δ) * l2sens)
    return (t .+ Distributions.rand(d, length(t)))
end

"""    truncatedLaplaceMech(x,f,l1sens::Real,ϵ::Real,δ::Real)
Computes a randomized version to f(x) according to the
truncated Laplace mechanism, for (ϵ,δ)-differential privacy.
This scheme adds bounded noise.
l1sens is the l1-sensitivity of f, which must be computed and provided by the
user. f must take values in R^k, for some k, i.e., return an array of k real
values.

Returns: (r, a) where r is a noisy version of f(x), and a defines the
support of the noise distribution (in the interval [f(x)-a, f(x)+a])
"""
function truncatedLaplaceMech(x, f, l1sens::Real, ϵ::Real, δ::Real)
    t = f(x)
    k = length(t)
    λ = l1sens/ϵ
    a = λ * log( 1 + exp(ϵ) * (k*(1-exp(-ϵ/k)))/(2*δ) )
    # alternative, more conservative but indpt of k
    # a = λ * log(1 + ϵ * exp(ϵ) / (2*δ))
    d1 = Distributions.Laplace(0, λ)
    d = Distributions.truncated(d1, -a, a)
    return (t .+ Distributions.rand(d, length(t)), a)
end

truncatedLaplaceMech

dpkf.jl:-

In [2]:
import Pkg

if haskey(Pkg.installed(),"MosekTools")
    using MosekTools
    const dpkf_ok = true
    println("MosekTools.jl found. The functions for differentially private Kalman filtering can be used.")
else
    dpkf_ok = false
    println("WARNING: MosekTools.jl not installed, the functions
    for differentially private Kalman filtering cannot be used!")
end

#=
if haskey(Pkg.installed(),"SCS")
    using SCS  # does not work with SCS currently for some reason
    const dpkf_ok = true
else
    dpkf_ok = false
    println("WARNING: SCS.jl not installed, the functions for differentially
    private Kalman filtering cannot be used!")
end
=#

using JuMP
using ControlSystems
using LinearAlgebra


"""    (D, P_val, X_val, Ω_val, M) =
            staticInputBlock_DPKF_ss(Ls, As, Cs, Winvs, Vs, Vinvs, k_priv, ρ,
                                    m=size(As,1), p=size(Cs,1), r=size(Ls,1))

Compute an input matrix D for the two-block differentially private
steady-state Kalman filter mechanism. D takes linear combinations of
individual signals before privacy-preserving Gaussian noise injection.

Note that we do not guarantee observability of the pair (A,D*C). If
observability is lost and needed, call the dfactor function with the
returned matrix M and a truncation threshold lower than 1e-6 in order
to compute a new matrix D with more rows.

Inputs: one matrix per individual

- Ls: size(r,nusers*m), or size (r,m,nusers) if want estimator of sum_i L_i x_i
- As: size (m,m,nusers)
- Cs: size (p,m,nusers)
- Winvs: size (m,m,nusers) -- inverses of process noise cov. matrices
- Vs: size (p,p,nusers) -- measurement noise cov. matrices
- Vinvs: inverses of Vs (computed externally for modularity/efficiency)
- ρ: vector of size nusers, for the adjacency definition: ||yᵢ-yᵢ'||₂ <= ρᵢ
for user i.
- k_priv: proportionality constant for Gaussian privacy-preserving noise
injection. Compute it with k_priv = gaussianMechConstant(ϵ,δ) for your
choice of ϵ, δ.
"""

function staticInputBlock_DPKF_ss(Ls, As, Cs, Winvs, Vs, Vinvs, ρ, k_priv,
                                  nusers=size(As,3), m=size(As,1),
                                  p=size(Cs,1), r=size(Ls,1))

    modl = Model(Mosek.Optimizer)
    #modl = Model(with_optimizer(Mosek.Optimizer))
    #modl = Model(with_optimizer(SCS.Optimizer))

    Nm = nusers * m
    Np = nusers * p

    A = zeros(Nm, Nm)
    C = zeros(Np, Nm)
    V = zeros(Np, Np)
    Winv = zeros(Nm, Nm)
    Vinv = zeros(Np, Np)

    for i=1:nusers
        A[(i-1)*m+1:i*m, (i-1)*m+1:i*m] = As[:,:,i]
        C[(i-1)*p+1:i*p, (i-1)*m+1:i*m] = Cs[:,:,i]
        V[(i-1)*p+1:i*p, (i-1)*p+1:i*p] = Vs[:,:,i]
        Winv[(i-1)*m+1:i*m, (i-1)*m+1:i*m] = Winvs[:,:,i]
        Vinv[(i-1)*p+1:i*p, (i-1)*p+1:i*p] = Vinvs[:,:,i]
    end

    if size(Ls,3) > 1
        L = zeros(r, Nm)
        for i=1:nusers
            L[:, (i-1)*m+1:i*m] = Ls[:,:,i]
        end
    else
        L = copy(Ls)
    end

    if r == 1
        @variable(modl, X >= 0)
        @objective(modl, Min, X)
    else
        @variable(modl, X[1:r, 1:r], PSD)
        @objective(modl, Min, tr(X))
    end
    if Nm == 1
        @variable(modl, Ω >= 0)
    else
        @variable(modl, Ω[1:Nm, 1:Nm], PSD)
    end
    if Np == 1
        @variable(modl, P >= 0)
    else
        @variable(modl, P[1:Np, 1:Np], PSD)
    end

    @SDconstraint(modl, hvcat((2,2), X, L, L', Ω) >= zeros(r+Nm,r+Nm))

    @SDconstraint(modl, hvcat((2,2),
        C'*P*C-Ω+Winv, Winv*A, A'*Winv, Ω+A'*Winv*A) >= zeros(2*Nm,2*Nm))

    for i=1:nusers
        e = zeros(p,Np); e[:,(i-1)*p+1:i*p]=Matrix{Float64}(I,p,p)
        @SDconstraint(modl,
          hvcat((2,2), Matrix{Float64}(I,p,p)/(k_priv^2*ρ[i]^2)+Vinvs[:,:,i], e, e', V-V*P*V)
          >= zeros(Np+p,Np+p))
    end

    optimize!(modl)
    status = termination_status(modl)

    println("============================================")
    println("Solution status: ", status)
    println("Objective value: ", objective_value(modl))
    println("============================================")

    P_val = value.(P)

    X_val = value.(X)
    Ω_val = value.(Ω)

    #= # Debug
    Σ_val = inv(Ω_val)
    println("Norm of the difference between Σ^{-1} and (A Σ A' + W)^{-1}+C' P C")
    E1 = C'*P_val*C+inv(A*Σ_val*A'+inv(Winv))-inv(Σ_val)
    println(norm(E1))
    =#

    # Matrix to factorize
    M = k_priv^2 * (inv(V-V*P_val*V)-Vinv)
    # Compute D matrix
    D = dfactor(M; svaltol=1e-6)

    # We return M also, so that the factorization can be redone if needed
    return (D, P_val, X_val, Ω_val, M)
end

"""    (D, P_val, X_val, Ω_val, M) =
            staticInputBlock_DPLQG_ss(Ls, As, Bs, Cs, Vs, Vinvs, Winvs, k_priv, ρ,
                                    m=size(As,1), p=size(Cs,1), r=size(Ls,1))

Compute an input matrix D for the two-block differentially private
steady-state LQG mechanism. D takes linear combinations of
individual signals before privacy-preserving Gaussian noise injection.

Note that we do not guarantee observability of the pair (A,D*C). If
observability is lost and needed, call the dfactor function with the
returned matrix M and a truncation threshold lower than 1e-6 in order
to compute a new matrix D with more rows.

Inputs: one matrix per individual

- Q: size(nusers*m,nusers*m) (m = dim of individual state-space)
- R: size(du,du) (du = dim of control input signal)
- As: size (m,m,nusers)
- Bs: size (m,du,nsusers) - All B matrices pre-multiply the same input u
- Cs: size (p,m,nusers)
- Vs: size (p,p,nusers)
- Vinvs: inverses of Vs (computed externally for modularity/efficiency)
- Winvs: size (m,m,nusers)
- ρ: vector of size nusers, for the adjacency definition: ||yᵢ-yᵢ'||₂ <= ρᵢ
for user i.
- k_priv: proportionality constant for Gaussian privacy-preserving noise
injection. Compute it with k_priv = gaussianMechConstant(ϵ,δ) for your
choice of ϵ, δ.
"""
function staticInputBlock_DPLQG_ss(Q, R, As, Bs, Cs, Ws, Winvs, Vs, Vinvs,
                                ρ, k_priv, nusers=size(As,3), m=size(As,1),
                                p=size(Cs,1), du=size(Bs,2))

    Nm = nusers * m

    A = zeros(Nm, Nm)
    B = zeros(Nm, du)  # the same control input is shared by all agents
    W = zeros(Nm, Nm)

    for i=1:nusers
        A[(i-1)*m+1:i*m, (i-1)*m+1:i*m] = As[:,:,i]
        B[(i-1)*m+1:i*m, :] = Bs[:,:,i]  # stack the Bs in one column
        W[(i-1)*m+1:i*m, (i-1)*m+1:i*m] = Ws[:,:,i]
    end

    P = dare(A, B, Q, R)  #  computes ss LQR cost
    N = A'*P*A+Q-P
    L = dfactor(N, svaltol=1e-5)  # factorize L' L = P
    r = size(L,1)

    (D, perf_val, X_val, Ω_val, M) =
        staticInputBlock_DPKF_ss(L, As, Cs, Winvs, Vs, Vinvs, ρ, k_priv, nusers, m, p, r)
    return (D, tr(P*W)+tr(X_val), P, X_val, M)
end



"""    dfactor(M; svaltol=1e-4)

Compute D such that D'D = M for M positive semidefinite, via SVD. Tries to
output a wide D matrix by neglecting the singular values of M smaller than
svaltol*(max svals).
"""
function dfactor(M; svaltol=1e-4)
    (U,S,V) = svd(M)
    svalmax = maximum(S)
    threshold = svaltol*svalmax
    cols = []
    for i=1:length(S)
        if S[i] >= threshold
            push!(cols, i)
        end
    end
    D = zeros(size(M,1), length(cols))
    for j=1:length(cols)
        D[:,j] = U[:,cols[j]] * sqrt(S[cols[j]])
    end
    return sign(D[1,1])*D'
end


"""    evaluateKFperf(D, Ls, As, Cs, Vs, Ws, ρ, k_priv)

Compute the steady-state MSE of the two-block differentially private
Kalman filter, with a static matrix D to combine input signals.
The performance is computed by solving an algebraic Riccati equation.
It corresponds to the MSE at the end of a measurement update state in
the Kalman filter.

Inputs:
- D: size (q,nusers*p)  (q can be arbitrary, p = individual output signal dim.)
one matrix per individual
- Ls: size (r,m,nusers)  (if want estimator of sum_i L_i x_i)
- As: size (m,m,nusers)
- Cs: size (p,m,nusers)
- Ws: size (m,m,nusers) -- process noise cov. matrix
- Vs: size (p,p,nusers) -- measurement noise cov. matrix
- ρ: vector of size nusers, for the adjacency definition: ||yᵢ-yᵢ'||₂ <= ρᵢ
for user i.
- k_priv: proportionality constant for Gaussian privacy-preserving noise
injection. Compute it with k_priv = gaussianMechConstant(ϵ,δ) for your
choice of ϵ, δ.

Outputs: (performance, Σ, Σupdate)
where Σ and Σupdate are the error covariance matrices after the
prediction and measurement update step respectively
"""
function evaluateKFperf(D, Ls, As, Cs, Ws, Vs, ρ, k_priv)
    nusers=size(As,3)
    m=size(As,1)
    p=size(Cs,1)
    r=size(Ls,1)

    Nm = nusers * m
    Np = nusers * p

    L = zeros(r, Nm)
    A = zeros(Nm, Nm)
    C = zeros(Np, Nm)
    W = zeros(Nm, Nm)
    V = zeros(Np, Np)

    for i=1:nusers
        L[:, (i-1)*m+1:i*m] = Ls[:,:,i]
        A[(i-1)*m+1:i*m, (i-1)*m+1:i*m] = As[:,:,i]
        C[(i-1)*p+1:i*p, (i-1)*m+1:i*m] = Cs[:,:,i]
        W[(i-1)*m+1:i*m, (i-1)*m+1:i*m] = Ws[:,:,i]
        V[(i-1)*p+1:i*p, (i-1)*p+1:i*p] = Vs[:,:,i]
        #Winv[(i-1)*m+1:i*m, (i-1)*m+1:i*m] = Winvs[:,:,i]
        #Vinv[(i-1)*p+1:i*p, (i-1)*p+1:i*p] = Vinvs[:,:,i]
    end

    # compute sensitivity
    Δ₂ = 0
    for i=1:nusers
        Δ₂ = max(Δ₂, ρ[i] * maximum(svdvals(D[:,(i-1)*p+1:i*p])))
    end

    V₁ = D*V*D' + k_priv^2  * Δ₂^2 * Matrix{Float64}(I,size(D,1),size(D,1))
    C₁ = D*C
    V₁ = 0.5*(V₁+V₁')  # might have lost perfect symmetry due to numerical issues
    Σ = dare(A', C₁', W, V₁)  #  computes ss cov. after time update step
    Σupdate = Σ - Σ*C₁'*inv(C₁*Σ*C₁'+V₁)*C₁*Σ  # cov. afer meas. update step

    #return (trace(L*Σupdate*L'), Δ₂, Σupdate)
    return tr(L*Σupdate*L'), Σ, Σupdate
end




"""    evaluateLQGperf(D, Q, R, As, Bs, Cs, Ws, Vs, ρ, k_priv)

Compute the steady-state MSE of the two-block differentially private
LQG controller, with a static matrix D to combine input signals.
The performance is computed by solving two algebraic Riccati equations.

Inputs:
- D: size (q,nusers*p)  (q can be arbitrary, p = individual output signal dim.)
one matrix per individual
- Q: size(nusers*m,nusers*m) (m = dim of individual state-space)
- R: size(du,du) (du = dim of control input signal)
- As: size (m,m,nusers)
- Bs: size (m,du,nsusers) - All B matrices pre-multiply the same input u
- Cs: size (p,m,nusers)
- Ws: size (m,m,nusers) -- Process noise
- Vs: size (p,p,nusers) -- Measurement noise
- ρ: vector of size nusers, for the adjacency definition: ||yᵢ-yᵢ'||₂ <= ρᵢ
for user i.
- k_priv: proportionality constant for Gaussian privacy-preserving noise
injection. Compute it with k_priv = gaussianMechConstant(ϵ,δ) for your
choice of ϵ, δ.

Outputs: (performance, Σ, Σupdate, P)
where Σ and Σupdate are the error covariance matrices after the
prediction and measurement update step respectively, P is the solution
of the control algebraic Riccati equation
"""
function evaluateLQGperf(D, Q, R, As, Bs, Cs, Ws, Vs, ρ, k_priv)
    nusers=size(As,3)
    m=size(As,1)
    p=size(Cs,1)
    du=size(R,1)

    Nm = nusers * m
    Np = nusers * p

    A = zeros(Nm, Nm)
    B = zeros(Nm, du)  # the same control input is shared by all agents
    C = zeros(Np, Nm)
    W = zeros(Nm, Nm)
    V = zeros(Np, Np)

    for i=1:nusers
        A[(i-1)*m+1:i*m, (i-1)*m+1:i*m] = As[:,:,i]
        B[(i-1)*m+1:i*m, :] = Bs[:,:,i]  # stack the Bs in one column
        C[(i-1)*p+1:i*p, (i-1)*m+1:i*m] = Cs[:,:,i]
        W[(i-1)*m+1:i*m, (i-1)*m+1:i*m] = Ws[:,:,i]
        V[(i-1)*p+1:i*p, (i-1)*p+1:i*p] = Vs[:,:,i]
    end

    P = dare(A, B, Q, R)  #  computes ss LQR cost matrix

    # compute sensitivity
    Δ₂ = 0
    for i=1:nusers
        Δ₂ = max(Δ₂, ρ[i] * maximum(svdvals(D[:,(i-1)*p+1:i*p])))
    end

    V₁ = D*V*D' + k_priv^2  * Δ₂^2 * Matrix{Float64}(I,size(D,1),size(D,1))
    C₁ = D*C
    V₁ = 0.5*(V₁+V₁')  # might have lost perfect symmetry due to numerical issues
    Σ = dare(A', C₁', W, V₁)  #  computes ss cov. after time update step
    Σupdate = Σ - Σ*C₁'*inv(C₁*Σ*C₁'+V₁)*C₁*Σ  # cov. afer meas. update step

    N = Q+A'*P*A-P
    return tr(P*W + N*Σupdate), P, Σ, Σupdate
end

┌ Warning: Pkg.installed() is deprecated
└ @ Pkg /cache/build/default-amdci4-6/julialang/julia-release-1-dot-8/usr/share/julia/stdlib/v1.8/Pkg/src/Pkg.jl:675


MosekTools.jl found. The functions for differentially private Kalman filtering can be used.


evaluateLQGperf

In [3]:
@doc staticInputBlock_DPKF_ss

No documentation found.

`staticInputBlock_DPKF_ss` is a `Function`.

```
# 5 methods for generic function "staticInputBlock_DPKF_ss":
[1] staticInputBlock_DPKF_ss(Ls, As, Cs, Winvs, Vs, Vinvs, ρ, k_priv) in Main at /home/sambhaw/cps/experiments.ipynb:57
[2] staticInputBlock_DPKF_ss(Ls, As, Cs, Winvs, Vs, Vinvs, ρ, k_priv, nusers) in Main at /home/sambhaw/cps/experiments.ipynb:57
[3] staticInputBlock_DPKF_ss(Ls, As, Cs, Winvs, Vs, Vinvs, ρ, k_priv, nusers, m) in Main at /home/sambhaw/cps/experiments.ipynb:57
[4] staticInputBlock_DPKF_ss(Ls, As, Cs, Winvs, Vs, Vinvs, ρ, k_priv, nusers, m, p) in Main at /home/sambhaw/cps/experiments.ipynb:57
[5] staticInputBlock_DPKF_ss(Ls, As, Cs, Winvs, Vs, Vinvs, ρ, k_priv, nusers, m, p, r) in Main at /home/sambhaw/cps/experiments.ipynb:57
```


In [4]:
n = 10; m=2; p=2; r=1
Ls = zeros(r,m,n)
As = zeros(m,m,n)
Cs = zeros(p,m,n)
Vs = zeros(p,p,n)
Vinvs = zeros(p,p,n)
Ws = zeros(m,m,n)
Winvs = zeros(m,m,n)

for k=1:n
    Ls[:,:,k] = [1.0 0.0]
    As[:,:,k] = [-0.4 0.5; 0.4 0.7]
    Cs[:,:,k] = [1.0 0.0; 0.0 0.4]
    Ws[:,:,k] = 0.1*[1.0 0.2; 0.2 2]
    Winvs[:,:,k] = inv(Ws[:,:,k])
    Vs[:,:,k] = [0.3 0.0; 0.0 0.4]
    Vinvs[:,:,k] = inv(Vs[:,:,k])
end

k_priv = gaussianMechConstant(log(5), 0.05)
ρ = 1.0 * ones(n); ρ[1] = ρ[2] = 0.5;

In [6]:
(D, P_val, X_val, Ω_val, M) = staticInputBlock_DPKF_ss(Ls, As, Cs, Vs, Vinvs, Winvs, ρ, k_priv)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3581            
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 14              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.01    
Problem


  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3581            
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 14              
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 3581
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables       : 1                 conic                  : 0               
Optimizer  - Semi-definite variables: 14                scalarized             : 4001            
Factor     - setup time             : 0.29              dense det. time        : 0.00            
Factor     - ML order time          : 0.10              GP order time          : 0.00            
Factor     - nonzeros before factor : 3.20e+

2   1.7e+00  1.5e-01  1.6e-01  8.12e-01   5.457557590e+00   6.211366884e+00   1.5e-01  1.15  
3   8.4e-01  7.2e-02  3.5e-02  1.21e+00   1.129662979e+01   1.138860982e+01   7.2e-02  1.35  


4   5.4e-01  4.6e-02  1.9e-02  1.01e+00   1.374647995e+01   1.382209118e+01   4.6e-02  1.59  
5   1.1e-01  9.0e-03  1.5e-03  1.17e+00   1.780546063e+01   1.781590966e+01   9.0e-03  1.82  


6   9.6e-03  8.2e-04  4.3e-05  1.09e+00   1.882363528e+01   1.882491263e+01   8.2e-04  2.07  
7   3.2e-04  2.7e-05  2.7e-07  1.02e+00   1.892701180e+01   1.892706186e+01   2.7e-05  2.30  


8   1.7e-05  1.4e-06  3.2e-09  9.07e-01   1.893005311e+01   1.893005508e+01   1.4e-06  2.57  
9   7.2e-06  6.1e-07  9.9e-10  8.13e-01   1.892987008e+01   1.892987132e+01   6.1e-07  2.77  


10  9.4e-07  8.0e-08  4.7e-11  8.65e-01   1.892980482e+01   1.892980498e+01   8.0e-08  3.04  
11  9.9e-08  8.5e-09  1.9e-12  1.00e+00   1.892980585e+01   1.892980588e+01   8.4e-09  3.27  


12  3.5e-08  1.3e-09  1.0e-13  1.02e+00   1.892980540e+01   1.892980541e+01   1.2e-09  3.49  
Optimizer terminated. Time: 3.50    

Solution status: OPTIMAL


Objective value: 18.929805401283463


([3.929602091001504 -0.7096311384690989 … 3.877043776938252 -0.728303153851476; -0.5368804027618267 -2.747316772953508 … -0.4952549983420667 -2.691408781308557; … ; 0.3553046848958504 1.818293509254313 … 1.8605649138131356e-8 9.410168145580622e-8; -0.42389185384892025 -1.4639712253034078 … 0.1107433294852789 0.37435655697037934], [0.19319510121318004 0.02290989851124719 … 0.007602245337489882 -0.004781224727689811; 0.02290989851124719 0.196394425930143 … -0.004791071803154753 0.018635201504578022; … ; 0.007602245337489882 -0.004791071803154753 … 0.0354948048701667 0.0011287982695892518; -0.004781224727689811 0.018635201504578022 … 0.0011287982695892518 0.0449703678804968], 18.929805401283463, [0.3542671442539542 -0.02695939965266597 … 0.010413186778635446 -0.0025335874295290797; -0.02695939965266597 0.19174900126526181 … -0.002539358778036966 0.005261291514965333; … ; 0.010413186778635446 -0.002539358778036966 … 0.17865119153450187 -0.031128584602715285; -0.0025335874295290797 0.005261

In [7]:
D

20×20 Matrix{Float64}:
  3.9296       -0.709631     3.9296       …   3.87704     -0.728303
 -0.53688      -2.74732     -0.53688         -0.495255    -2.69141
 -2.82527e-8    5.68373e-9  -2.67953e-8       0.438613    -0.082955
  1.32237e-8   -2.81766e-9   1.11542e-8      -0.363079     0.0686693
  2.32982e-8   -4.57043e-9   2.82083e-8      -0.890617     0.168443
 -1.74432e-8    3.72551e-9  -1.94798e-9   …   2.82944     -0.535134
  2.26768e-8   -4.76257e-9   1.55913e-8       1.99177     -0.376704
 -2.26119e-8    4.84114e-9  -1.58429e-8      -0.628384     0.118846
  2.17168e-8   -4.86631e-9   2.21258e-8       0.459702    -0.0869437
  2.7554       -0.53842     -2.7554          -7.46497e-9   3.28691e-9
 -2.3324        0.683493    -2.3324       …   0.590938    -0.166847
 -5.60391e-10   9.35336e-9  -6.23812e-10      0.0591492    0.312743
  1.27427e-10  -5.72788e-9   1.34192e-10     -0.0442554   -0.233994
 -8.71594e-10  -1.01842e-8  -8.71892e-10     -0.00472545  -0.0249851
 -5.82348e-10   5.647

In [8]:
evaluateKFperf(D, Ls, As, Cs, Vs, Ws, ρ, k_priv)

(2.2183469840382894, [0.5478385716770382 0.28972302182391774 … -0.009012764520054513 -0.010861501467933767; 0.28972302182391774 1.1433974673775484 … -0.010873418628518322 -0.03734169392703811; … ; -0.009012764520054513 -0.010873418628518322 … 0.5458521369212734 0.28580907692493185; -0.010861501467933767 -0.03734169392703811 … 0.28580907692493185 1.1312788617897622], [0.45349024170742724 0.24351932144108784 … -0.02666864610602332 -0.017700642510739405; 0.24351932144108784 1.0907514463211254 … -0.01767581509285404 -0.04728429065523812; … ; -0.026668646106023312 -0.017675815092854052 … 0.448007416195366 0.2367727771961869; -0.017700642510739408 -0.04728429065523813 … 0.23677277719618686 1.07552024483396])

In [9]:
evaluateKFperf(eye(2*n), Ls, As, Cs, Vs, Ws, ρ, k_priv)

UndefVarError: UndefVarError: eye not defined